In [107]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [108]:
from skforecast.ForecasterBaseline import ForecasterLastEquivalentDate
from skforecast.model_selection import backtesting_forecaster
import numpy as np
import pandas as pd
from skforecast import __version__
print(__version__)


0.10.1


In [109]:
# simulate a time series
n = 365
time = pd.date_range(start='2000-01-01', periods=n, freq='d')
y = np.arange(n)
df = pd.DataFrame({'time': time, 'y': y})
df = df.set_index('time')
df = df.asfreq('d')
df

,y
time,
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
...,...
2000-12-26,360
2000-12-27,361
2000-12-28,362


In [110]:
forecaster = ForecasterLastEquivalentDate(offset=10, n_equivalent_dates=2)
forecaster.fit(y=df['y'])
forecaster

ForecasterLastEquivalentDate 
Offset: 10 
Number of equivalent dates: 2 
Aggregation function: mean 
Window size: 20 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-10-26 23:10:44 
Last fit date: 2023-10-26 23:10:44 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [111]:
forecaster.last_window

time
2000-12-11    345
2000-12-12    346
2000-12-13    347
2000-12-14    348
2000-12-15    349
2000-12-16    350
2000-12-17    351
2000-12-18    352
2000-12-19    353
2000-12-20    354
2000-12-21    355
2000-12-22    356
2000-12-23    357
2000-12-24    358
2000-12-25    359
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Name: y, dtype: int32

In [112]:
forecaster.predict(steps=3)

2000-12-31    350.0
2001-01-01    351.5
2001-01-02    353.0
Freq: D, Name: pred, dtype: float64

In [120]:
metric, predictions = backtesting_forecaster(
    forecaster=forecaster,
    y=df['y'],
    initial_train_size=int(len(df) * 0.5),
    steps=9,
    metric='mean_absolute_error',
    verbose=True
)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 182
Number of observations used for backtesting: 183
    Number of folds: 21
    Number of steps per fold: 9
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-01 00:00:00 -- 2000-07-09 00:00:00  (n=9)
Fold: 1
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-10 00:00:00 -- 2000-07-18 00:00:00  (n=9)
Fold: 2
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-19 00:00:00 -- 2000-07-27 00:00:00  (n=9)
Fold: 3
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-28 00:00:00 -- 2000-08-05 00:00:00  (n=9)
Fold: 4
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)

  0%|          | 0/21 [00:00<?, ?it/s]

In [121]:
predictions

,pred
2000-07-01,167.0
2000-07-02,168.5
2000-07-03,170.0
2000-07-04,171.5
2000-07-05,173.0
...,...
2000-12-26,348.5
2000-12-27,350.0
2000-12-28,347.0
2000-12-29,348.5


In [115]:
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.offsets import DateOffset
ts = pd.Timestamp('2017-01-01 09:10:11')
ts + DateOffset(months=1)

Timestamp('2017-02-01 09:10:11')

In [116]:
offset = offsets.Day()
ts = pd.Timestamp('2017-01-01 09:10:11')
ts - offset

Timestamp('2016-12-31 09:10:11')

In [117]:
y = np.arange(365)
steps = 3
offset = 10
agg_function = np.mean
n_equivalent_dates = 2
equivalent_indexes = np.tile(
                        np.arange(-offset, 0),
                        int(np.ceil(steps/offset))
                     )
equivalent_indexes = equivalent_indexes[:steps]
equivalent_indexes = [equivalent_indexes * n for n in np.arange(n_equivalent_dates, 0, -1)]
equivalent_indexes = np.vstack(equivalent_indexes)
# extract from y the values of indexes in offset_indexes
equivalent_values = y[equivalent_indexes]
# apply agg_function to equivalent_values by column
equivalent_values = np.apply_along_axis(agg_function, axis=0, arr=equivalent_values)
equivalent_values

array([350. , 351.5, 353. ])

In [118]:
y = np.arange(365)
steps = 3
offset = 10
agg_function = np.mean
n_equivalent_dates = 2
equivalent_indexes = np.tile(
                        np.arange(-offset, 0),
                        int(np.ceil(steps/offset))
                     )
equivalent_indexes = equivalent_indexes[:steps]
equivalent_indexes = [equivalent_indexes * n for n in np.arange(n_equivalent_dates, 0, -1)]
equivalent_indexes = np.vstack(equivalent_indexes)
equivalent_values  = y[equivalent_indexes]
np.mean(equivalent_values, axis=0)

array([350. , 351.5, 353. ])

In [119]:
equivalent_indexes = np.column_stack(equivalent_indexes)
equivalent_indexes

array([[-20, -10],
       [-18,  -9],
       [-16,  -8]])